### Analyse data Titanic Disaster

In [1]:
# setup absolute path to location of package Starts and config-file 
from inspect import getsourcefile
import os.path as path, sys
current_dir = path.dirname(path.abspath(getsourcefile(lambda:0)))
sys.path.insert(0, current_dir[:current_dir.rfind(path.sep)])

from Starts.startmod import *
# from Starts.startvis import *  # Commented because of deprecated basemap-package
%matplotlib inline

local_kwargs {'data_path': './data/train.csv,./data/test.csv', 'drop_obj_col': False, 'nan_drop_col': False, 'nan_drop_row': False, 'nan_zero': False, 'nan_mean': True, 'nan_mean_neighbors': False}


In [2]:
train_data = idata[0]
attributes=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Dr', 'Ms', 'Capt']
train_data = StartMod.feature_engineering(train_data, old_feature='Name', new_feature='Title', new_attributes=attributes, 
                                          rm=True)
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [3]:
train_data = StartMod.feature_engineering_merge_cols(train_data, ['SibSp', 'Parch'], 'FamilySize')
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked,Title,FamilySize
0,1,0,3,male,22.0,A/5 21171,7.2500,NaN,S,Mr,1
1,2,1,1,female,38.0,PC 17599,71.2833,C85,C,Mrs,1
2,3,1,3,female,26.0,STON/O2. 3101282,7.9250,NaN,S,Miss,0
3,4,1,1,female,35.0,113803,53.1000,C123,S,Mrs,1
4,5,0,3,male,35.0,373450,8.0500,NaN,S,Mr,0


In [4]:
# check NaN columns in data
train_data.head()
StartMod.nan_columns(train_data)

['Age', 'Cabin', 'Embarked', 'Title']

In [5]:
# simplify data by removing the redundant feature_columns
train_data = StartMod.feature_selection(train_data, ['Pclass', 'Ticket', 'Cabin', 'Embarked'], 
                                        dependent_label='Survived', rm=False)
train_data.head()

Data might have NaN_value, please clean them before building model


,PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked,Title,FamilySize
0,1,0,3,male,22.0,A/5 21171,7.2500,NaN,S,Mr,1
1,2,1,1,female,38.0,PC 17599,71.2833,C85,C,Mrs,1
2,3,1,3,female,26.0,STON/O2. 3101282,7.9250,NaN,S,Miss,0
3,4,1,1,female,35.0,113803,53.1000,C123,S,Mrs,1
4,5,0,3,male,35.0,373450,8.0500,NaN,S,Mr,0


In [6]:
# identify the index of data
StartMod.idx_columns(train_data)

[('PassengerId', 0, dtype('int64')),
 ('Survived', 1, dtype('int64')),
 ('Pclass', 2, dtype('int64')),
 ('Sex', 3, dtype('O')),
 ('Age', 4, dtype('float64')),
 ('Ticket', 5, dtype('O')),
 ('Fare', 6, dtype('float64')),
 ('Cabin', 7, dtype('O')),
 ('Embarked', 8, dtype('O')),
 ('Title', 9, dtype('float64')),
 ('FamilySize', 10, dtype('int64'))]

In [7]:
# check NaN columns in data
StartMod.nan_columns(train_data)

['Age', 'Cabin', 'Embarked', 'Title']

In [8]:
# process NaN feature columns in a simple way by replacing with 'Unknown'
train_data = StartMod.process_nan_simply(train_data, nan_column='Title')
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked,Title,FamilySize
0,1,0,3,male,22.0,A/5 21171,7.2500,NaN,S,Mr,1
1,2,1,1,female,38.0,PC 17599,71.2833,C85,C,Mrs,1
2,3,1,3,female,26.0,STON/O2. 3101282,7.9250,NaN,S,Miss,0
3,4,1,1,female,35.0,113803,53.1000,C123,S,Mrs,1
4,5,0,3,male,35.0,373450,8.0500,NaN,S,Mr,0


In [9]:
StartMod.nan_columns(train_data)

['Age', 'Cabin', 'Embarked']

In [10]:
# process NaN feature columns
train_data = StartMod.process_nan_columns(train_data)
train_data.head()

-> Replace NaN value by mean-value at column:  Age
-> Drop column:  Cabin
-> Drop column:  Embarked


,PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Title,FamilySize
0,1,0,3,male,22.0,A/5 21171,7.2500,Mr,1
1,2,1,1,female,38.0,PC 17599,71.2833,Mrs,1
2,3,1,3,female,26.0,STON/O2. 3101282,7.9250,Miss,0
3,4,1,1,female,35.0,113803,53.1000,Mrs,1
4,5,0,3,male,35.0,373450,8.0500,Mr,0


In [11]:
# categorize feature in obj type and non_obj (numeric) type
non_obj_features, obj_features = StartMod.feature_columns(train_data)
non_obj_features, obj_features

(['PassengerId', 'Survived', 'Pclass', 'Age', 'Fare', 'Title', 'FamilySize'],
 ['Sex', 'Ticket'])

In [12]:
StartMod.nan_columns(train_data)

['Age']

In [13]:
# drop column with NaN
func = lambda col: train_data.drop(col, axis=1)
train_data = func(['Age'])
train_data.head()

,PassengerId,Survived,Pclass,Sex,Ticket,Fare,Title,FamilySize
0,1,0,3,male,A/5 21171,7.2500,Mr,1
1,2,1,1,female,PC 17599,71.2833,Mrs,1
2,3,1,3,female,STON/O2. 3101282,7.9250,Miss,0
3,4,1,1,female,113803,53.1000,Mrs,1
4,5,0,3,male,373450,8.0500,Mr,0


In [14]:
StartMod.nan_columns(train_data)

[]

In [15]:
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Ticket', 'Fare', 'Title',
       'FamilySize'],
      dtype='object')

In [16]:
label_columns=['Title', 'Sex', 'Ticket']
for label in label_columns:
    print(label, train_data[label].dtype)


Title float64
Sex object
Ticket object


In [17]:
# encode the columns 'Sex' and 'Title' 
train_data = StartMod.encode_label_column(train_data, label_columns=['Title', 'Sex', 'Ticket'], one_hot=False)
train_data.head()

Type of label_column Title is float64


,PassengerId,Survived,Pclass,Sex,Ticket,Fare,Title,FamilySize
0,1,0,3,male,A/5 21171,7.2500,Mr,1
1,2,1,1,female,PC 17599,71.2833,Mrs,1
2,3,1,3,female,STON/O2. 3101282,7.9250,Miss,0
3,4,1,1,female,113803,53.1000,Mrs,1
4,5,0,3,male,373450,8.0500,Mr,0


In [18]:
train_data = train_data.drop('PassengerId', axis=1)
StartMod.idx_columns(train_data)

[('Survived', 0, dtype('int64')),
 ('Pclass', 1, dtype('int64')),
 ('Sex', 2, dtype('O')),
 ('Ticket', 3, dtype('O')),
 ('Fare', 4, dtype('float64')),
 ('Title', 5, dtype('float64')),
 ('FamilySize', 6, dtype('int64'))]

In [19]:
StartML.nan_columns(train_data)

[]

In [22]:
type(train_data)

dask.dataframe.core.DataFrame

In [18]:
from sklearn.utils import shuffle
train_data = shuffle(train_data)
train_data.to_csv('./data/titanic_no_nan.csv', index=False)